# Baidu Cloud Chinese Semantic Checker

This notebook is used to check the semantic of Chinese language unit. It is based on Baidu Cloud's API.

Author: Zexin Xu, Zilu Zhang

## Baidu Cloud API

This snippet invloves the Baidu API. `API_key` and `Secret_key` are deleted for security reasons. Please use your own API key and secret key. For more details of the usage of Baidu API, please refer to [Baidu Cloud API](https://cloud.baidu.com/doc/NLP/s/tk6z52czc). If you are a English user, there is a language switch button on the top right corner.

In [ ]:
import pandas as pd
import requests
import json

API_key = ""
Secret_key = ""

In [ ]:
def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

access_token = get_access_token()

In [ ]:
#NOTE Put your dataframes here, and modify the column names accordingly
baidu_df = pd.DataFrame()

In [ ]:
url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/ecnet?charset=UTF-8&access_token=" + access_token

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

for i, row in baidu_df.loc[:, :].iterrows():
    payload = json.dumps({
        "text": row['sentences']
    })
    response = requests.request("POST", url, headers=headers, data=payload).json()
    baidu_df.loc[i, "content"] = str(response['item']['vec_fragment'])
    if response['item']['score'] > 0:
        baidu_df.loc[i, "result"] = 0
    else:
        baidu_df.loc[i, "result"] = 1
    if i % 50 == 0:
        print(i, "iters done...")

baidu_df.to_csv('baidu/sen_df_result.csv', index=False, encoding='utf-8-sig')